In [1]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

print("Device being used: %s" %device)

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB
Device being used: cuda


In [2]:
path = '/scratch/kap9623/Final'
print(path)

/scratch/kap9623/Final


In [3]:
%%capture
!pip install simpletransformers

In [4]:
#retrieving paths for ann data

import glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from simpletransformers.ner import NERModel,NERArgs
# import wandb

ann_orig = glob.glob(path + '/ann_ref/*.ann' )
print(len(ann_orig))

389


# **Task 1: Name Entry Recognition for Medication Extraction**

# MODEL: SciBERT

## Note-level Split

### Reformatting from BRAT to ConLL

In [5]:
outfile = path + '/SciBERT/output_note_lvl.txt'
with open(outfile, 'w') as output_file:
  out_list = []
  for ann_file in ann_orig:
    with open(ann_file, 'r') as ann_in:
      txt_file = path + f'/ann_ref/{ann_file[-10:-4]}.txt'
      with open(txt_file, 'r') as txt_in:
        start_end = []
        
        #reading each line of .ann file and extracting start and end indices 
        for line in ann_in:
          entry = line.split()
          start_end.append([int(entry[2]), int(entry[3])])

        #reading each line of .txt file and extracting drug names using list of indices
        text_tokens = txt_in.read().split()
        drug_names = []
        for indicies in start_end:
          start = indicies[0]
          end = indicies[1]

          with open(txt_file, 'r') as txt_in:
            drug_names.append(txt_in.read()[indicies[0]:indicies[1]])
        
        #appending "O" or "DRUG" label to each txt token with relevant spacers
        for token in text_tokens:
          token_alnum = ''.join(ch for ch in token if ch.isalnum())
          if token_alnum in drug_names:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tDRUG\n')
          else:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t') #NOTE LEVEL
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tO\n')
        out_list.append('\n')
  out_string = ''.join(out_list)
  output_file.write(out_string)

### Reading ConLL output as dataframe

In [6]:
data = pd.read_csv(path + '/SciBERT/output_note_lvl.txt', sep='\t', lineterminator='\n', names = ['Note', 'Token', 'Label'])
data = data.fillna(method = 'ffill')

In [7]:
#adding token count for each token by note
token_count = []
for note in data['Note'].unique():
  df = data.loc[data['Note']== note]
  count = 0
  for token in df['Token']:
    token_count.append(count)
    count += 1

data['Token Count'] = token_count
data.sort_values(by = ['Note', 'Token Count'], inplace=True)
data.reset_index(drop = True, inplace = True)

In [8]:
#label encoding Note column
le = LabelEncoder()
note_en = le.fit_transform(data['Note'])
data['Note'] = note_en

In [9]:
#renaming columns to match model requirements
data.rename(columns = {'Note':'sentence_id', 'Token':'words', 'Label':'labels'}, inplace=True)

### Model Building

In [10]:
#splitting data into featues and labels
X = data[['sentence_id','words']]
y = data['labels']

#splitting into train, val, test (70-10-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, shuffle=False)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
valid_data = pd.DataFrame({"sentence_id":X_valid["sentence_id"],"words":X_valid["words"],"labels":y_valid})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [11]:
#checking unique token labels
label = data["labels"].unique().tolist()
label

['O', 'DRUG']

# Default performance (from previous run)

In [31]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.008105745364446193,
 'precision': 0.9607843137254902,
 'recall': 0.8909090909090909,
 'f1_score': 0.9245283018867925}

# Performance with freezing

In [32]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#freezing embedding layers for pretrained model
# print(model_sci_note.get_named_parameters())
emb_layers_list = ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias']

#set lr for embedding layers to 0
args.custom_parameter_groups = [
    {
        "params": emb_layers_list,
        "lr": 0,
    }
]

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.011424973296622435,
 'precision': 0.9565217391304348,
 'recall': 0.8,
 'f1_score': 0.8712871287128713}

# Performance with weighted loss

In [33]:
#calculating class weights based on imbalance
n_samples = len(train_data)
n_classes = 2
freq_O = train_data['labels'].value_counts()[0]
freq_DRUG = train_data['labels'].value_counts()[1]

weight_O = n_samples / (n_classes * freq_O) 
print(weight_O)

weight_DRUG = n_samples / (n_classes * freq_DRUG) 
print(weight_DRUG)

weights = [weight_O, weight_DRUG]

0.5125804829887884
20.372051035146846


In [35]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, weight = weights, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.1974275012811025,
 'precision': 0.9245283018867925,
 'recall': 0.8909090909090909,
 'f1_score': 0.9074074074074073}

# Performance with freezing + weighted loss

In [36]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#freezing embedding layers for pretrained model
# print(model_sci_note.get_named_parameters())
emb_layers_list = ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias']

#set lr for embedding layers to 0
args.custom_parameter_groups = [
    {
        "params": emb_layers_list,
        "lr": 0,
    }
]

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, weight = weights, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.3204003032296896,
 'precision': 0.94,
 'recall': 0.8545454545454545,
 'f1_score': 0.8952380952380952}

# Best Performance: No freezing/No weighted loss

### lr = 1e-4, bs = 8

In [22]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.017308803425112273,
 'precision': 0.9565217391304348,
 'recall': 0.8,
 'f1_score': 0.8712871287128713}

### lr = 1e-4, bs = 16

In [23]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.010434972494840622,
 'precision': 0.9038461538461539,
 'recall': 0.8545454545454545,
 'f1_score': 0.8785046728971962}

### lr = 1e-4, bs = 32

In [24]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.008105745364446193,
 'precision': 0.9607843137254902,
 'recall': 0.8909090909090909,
 'f1_score': 0.9245283018867925}

### lr = 1e-5, bs = 8

In [25]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.009753500297301798,
 'precision': 0.9387755102040817,
 'recall': 0.8363636363636363,
 'f1_score': 0.8846153846153846}

### lr = 1e-5, bs = 16

In [26]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.010556450788863003,
 'precision': 0.9574468085106383,
 'recall': 0.8181818181818182,
 'f1_score': 0.8823529411764706}

### lr = 1e-5, bs = 32

In [27]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.009672382613644004,
 'precision': 0.9583333333333334,
 'recall': 0.8363636363636363,
 'f1_score': 0.8932038834951457}

### lr = 1e-6, bs = 8

In [28]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.018553924467414617,
 'precision': 0.8857142857142857,
 'recall': 0.5636363636363636,
 'f1_score': 0.6888888888888889}

### lr = 1e-6, bs = 16

In [29]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.031181396916508675,
 'precision': 1.0,
 'recall': 0.14545454545454545,
 'f1_score': 0.25396825396825395}

### lr = 1e-6, bs = 32

In [12]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 1e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.04941436027487119,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

### lr = 5e-4, bs = 8

In [13]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.056546044908463954,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

### lr = 5e-4, bs = 16

In [14]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.056819812208414075,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

### lr = 5e-4, bs = 32

In [15]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-4
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.06011636431018511,
 'precision': 0.0,
 'recall': 0.0,
 'f1_score': 0.0}

### lr = 5e-5, bs = 8

In [16]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.016169359668037942,
 'precision': 0.9166666666666666,
 'recall': 0.8,
 'f1_score': 0.854368932038835}

### lr = 5e-5, bs = 16

In [17]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.011154550377978013,
 'precision': 0.9215686274509803,
 'recall': 0.8545454545454545,
 'f1_score': 0.8867924528301887}

### lr = 5e-5, bs = 32

In [18]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-5
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.00811569532379508,
 'precision': 0.96,
 'recall': 0.8727272727272727,
 'f1_score': 0.9142857142857144}

### lr = 5e-6, bs = 8

In [19]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 8
args.eval_batch_size = 8
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/34 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.009455590110155754,
 'precision': 0.9038461538461539,
 'recall': 0.8545454545454545,
 'f1_score': 0.8785046728971962}

### lr = 5e-6, bs = 16

In [20]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 16
args.eval_batch_size = 16
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.01076611182652414,
 'precision': 0.9772727272727273,
 'recall': 0.7818181818181819,
 'f1_score': 0.8686868686868686}

### lr = 5e-6, bs = 32

In [21]:
#setting hyperparameters to default for all models
args = NERArgs()
args.learning_rate = 5e-6
args.scheduler = 'constant_schedule'
args.overwrite_output_dir = True
args.manual_seed = 42
args.train_batch_size = 32
args.eval_batch_size = 32
args.num_train_epochs = 10

#instantiating model
model_sci_note = NERModel('bert', 'allenai/scibert_scivocab_cased',labels=label,args =args, use_cuda = True)

#training model on train data and tuning hyperparameters on valid data
model_sci_note.train_model(train_data,eval_data = valid_data)

# #reinstantiating NERModel using best epoch from training
# model_sci_note = NERModel('bert', 'outputs/best_model')

#evaluating on test data
msn_result, msn_outputs, msn_preds = model_sci_note.eval_model(test_data)

print('Model: SciBERT')
print('Evaluation metrics (Note-level split):')
msn_result

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

  0%|          | 0/1 [00:00<?, ?it/s]

/home/kap9623/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Model: SciBERT
Evaluation metrics (Note-level split):


/home/kap9623/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.015882685780525208,
 'precision': 0.9,
 'recall': 0.6545454545454545,
 'f1_score': 0.7578947368421052}